In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

# Exploration

In this first notebook we are going to explore the area of interest (AoI) we have selected for the workshop. Having our AoI, and within a range of dates we have defined, we are going to look for available Sentinel-2 imagery on the AoI through the [Sentinel Hub](https://www.sentinel-hub.com/) platform.

## Explore the AoI

First of all, let's explore the area of interest of this workshop. In this case we have chosen the [Boadella reservoir](https://es.wikipedia.org/wiki/Embalse_de_Darnius_Boadella) in Catalonia, Spain, which geometry is in the data folder as `data/boadella.geojson`. 

To do the exploration we are going to use [leafmap](https://leafmap.org/). As we want to make the first zoom to our AoI we need the centroid of the geoJSON file, which will also help us later to extract the bounding box of the images to download.

In [11]:
import leafmap
import geopandas as gpd

in_geojson = 'workshop_data/boadella.geojson'
gdf = gpd.read_file(in_geojson)

centroid_coords = gdf['geometry'].centroid
centroid = [centroid_coords.y.values[0], centroid_coords.x.values[0]]   # We are going to use the centroid later

m = leafmap.Map(center=centroid, zoom=13)
m.add_geojson(in_geojson, layer_name="Boadella reservoir")
m

Map(center=[42.347577325903515, 2.815024677909404], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Create bounding box

With the aim that all the images in the dataset have the same dimensions (512x512 pixels) we are going to use the centroid that we extracted before from the geoJSON and we are going to generate a bounding box of 512x512 pixels at 10m per pixel.

In [12]:
from eotdl.tools import bbox_from_centroid

boadella_bbox = bbox_from_centroid(x=centroid[0], y=centroid[1], pixel_size=10, width=512, height=512)

Let's visualize the bounding box on a map!

In [13]:
from eotdl.tools import bbox_to_polygon

# Create a polygon from the bbox
boadella_polygon = bbox_to_polygon(boadella_bbox)
# Create a GeoDataFrame from the polygon
gdf = gpd.GeoDataFrame(geometry=[boadella_polygon])
# Save the bounding box as a geoJSON file, if needed
gdf.to_file('workshop_data/boadella_bbox.geojson', driver='GeoJSON')   # Uncomment to save the bbox as a GeoJSON file

m.add_geojson('workshop_data/boadella_bbox.geojson', layer_name="Boadella bbox")
m

Map(center=[42.347577325903515, 2.815024677909404], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Explore Sentinel-2 imagery

Now that he have our desired bounding box we can look for available Sentinel-2 imagery on it. This can be done through the EOTDL. 

First, we can look for which Sentinel sensors are supported in the EOTDL

In [14]:
from eotdl.access import SUPPORTED_SENSORS

SUPPORTED_SENSORS

('sentinel-1-grd', 'sentinel-2-l1c', 'sentinel-2-l2a', 'dem')

If we want to look for available Sentinel-2 imagery in our AoI, we must define a range of dates in which to search for the images. We have already defined a time interval for this workshop, which is in the `data/dates.csv` file.

In [15]:
import csv

dates = list()
with open("workshop_data/dates.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        dates.append(row[0])
dates.sort()

dates[:5]

['2020-01-13', '2020-01-28', '2020-02-02', '2020-06-21', '2020-09-14']

Although we have the specific dates, we are going to search for the entire time interval, just as a demonstrator.

In [16]:
from eotdl.access import search_sentinel_imagery

time_interval = (dates[0], dates[-1])

r = search_sentinel_imagery(time_interval, boadella_bbox, 'sentinel-2-l2a')
response = list(r)
response[:5]

('2020-01-13', '2022-06-01')


[{'id': 'S2B_MSIL2A_20220601T103629_N0400_R008_T31TDG_20220601T135543',
  'properties': {'datetime': '2022-06-01T10:49:26Z', 'eo:cloud_cover': 0.23}},
 {'id': 'S2B_MSIL2A_20220601T103629_N0400_R008_T31TDH_20220601T135543',
  'properties': {'datetime': '2022-06-01T10:49:14Z', 'eo:cloud_cover': 12.82}},
 {'id': 'S2A_MSIL2A_20220527T103631_N0400_R008_T31TDG_20220527T183616',
  'properties': {'datetime': '2022-05-27T10:49:34Z', 'eo:cloud_cover': 85.6}},
 {'id': 'S2A_MSIL2A_20220527T103631_N0400_R008_T31TDH_20220527T183616',
  'properties': {'datetime': '2022-05-27T10:49:19Z', 'eo:cloud_cover': 30.42}},
 {'id': 'S2B_MSIL2A_20220522T103629_N0400_R008_T31TDG_20220522T124154',
  'properties': {'datetime': '2022-05-22T10:49:27Z', 'eo:cloud_cover': 12.99}}]

They make sense, as the [revisit time](https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/#basic-facts) for Sentinel-2 is 5 days.

As a final step, let's check the number of dates with available images.

In [17]:
print(len(response))

342


To sum up this section, we have explored our AoI, generated a bounding box and a time interval in which to look for imagery and searched for Sentinel-2 imagery.

Let's continue in the [01_download](./01_download.ipynb) notebook and download the images!